# Top 10 selector

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('../data/10%sample/2020-Jan.csv_10%.csv')

In [3]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-01-01 00:00:01 UTC,view,100063693,2053013552427434207,apparel.shirt,turtle,136.43,519046195,d1e2f343-84bb-49bd-b13d-ca0f1ed9910e
1,2020-01-01 00:00:04 UTC,view,3600666,2232732092297380188,appliances.kitchen.washer,samsung,321.73,556820148,8748d326-2623-42b7-b422-df03db98fa58
2,2020-01-01 00:00:11 UTC,cart,1201565,2232732101407408685,apparel.shoes.slipons,apple,385.34,581430108,7d7687c4-b613-4467-8a81-54c7600e0ca9
3,2020-01-01 00:00:11 UTC,view,22700725,2232732091643068746,NaN,ombra,342.09,513398384,991eff38-b6f4-4503-9a88-f41edfdd5912
4,2020-01-01 00:00:13 UTC,view,4801028,2232732079706079299,sport.bicycle,xiaomi,14.95,574723072,2f967c6e-b6c4-4971-8599-d34f86570e29


In [25]:
# df.query("brand == 'apple' and category_code == 'sport bicycle'")  #check the dataframe output
df.query("product_id == 1002544")

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
108,2020-01-01 00:07:22 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,445.95,572284245,b6cc0b20-2771-4411-9422-c0f912fb2818
764,2020-01-01 00:57:27 UTC,cart,1002544,2232732093077520756,construction.tools.light,apple,445.95,595422027,7fc19599-2f07-4bef-9827-1ef69e58af9c
1378,2020-01-01 01:43:52 UTC,cart,1002544,2232732093077520756,construction.tools.light,apple,445.95,592656387,5923a9ed-9f65-41d1-a9ed-e224ba34a871
1821,2020-01-01 02:11:43 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,445.95,529184865,9c74a89e-94e0-4809-8360-ecfbeb7ecd9a
1904,2020-01-01 02:16:30 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,445.95,524698102,6bf7f1b1-45c4-4b1c-950a-48a4860276cc
...,...,...,...,...,...,...,...,...,...
5598848,2020-01-31 23:32:37 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,410.42,516671531,8633b8db-d817-4582-b347-0a3c80255092
5598875,2020-01-31 23:34:02 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,410.42,535039620,2d67937c-a688-4966-9113-9535eb5c8321
5598885,2020-01-31 23:34:32 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,410.42,608818031,18981178-3141-4701-8705-7e1d6954cf6e
5598951,2020-01-31 23:37:59 UTC,cart,1002544,2232732093077520756,construction.tools.light,apple,410.42,567888225,c1d7451f-a16b-4452-b728-6a55231e8705


In [5]:
def preprocessing_feat(X):    
    X_preprocessed = X.dropna(subset = ['category_code', 'brand']) #tbd!!
    X_preprocessed = X_preprocessed.drop_duplicates()
    X_preprocessed['category_code'] = X_preprocessed['category_code'].str.replace('.',' ')
    return X_preprocessed

def pricing_criterion(X):
    pricing_guide = X.groupby('category_code')['price'].describe()[["25%", "75%"]].reset_index()
    X_merged = X.merge(pricing_guide, on="category_code", how="right")
    X_merged["price_category"] = X_merged.apply(lambda row: make_column(row), axis=1)
    return X_merged

def make_column(row):
    if row["price"] < row["25%"]:
        return "low"
    elif row["price"] < row["75%"]:
        return "medium"
    else:
        return "high"

def metadata(X):
    X['metadata'] = X[['category_code', 'brand', 'price_category']].apply(lambda x: ' '.join(x), axis = 1)
    return X

def rating(X):
    dct = {'view': 1, 'cart': 3, 'purchase': 5}
    X['rating'] = X['event_type'].map(dct)
    return X

In [6]:
X_prep = preprocessing_feat(df)

/var/folders/6f/vn_j9pqn1qd8rp92y6x0h0hh0000gn/T/ipykernel_22720/3456390047.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X_preprocessed['category_code'] = X_preprocessed['category_code'].str.replace('.',' ')


In [7]:
# X_prep.query("brand == 'apple' and category_code == 'sport bicycle'")  #check the dataframe output
X_prep.query("product_id == 4804718")

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
42,2020-01-01 00:02:25 UTC,view,4804718,2232732079706079299,sport bicycle,apple,341.06,513814816,3d6aaddf-76a7-480f-af54-79b3a8ba14c4
203,2020-01-01 00:13:17 UTC,view,4804718,2232732079706079299,sport bicycle,apple,341.06,537104810,3c3c2ddd-a1ec-46cd-8d21-894872ce9be4
260,2020-01-01 00:17:30 UTC,view,4804718,2232732079706079299,sport bicycle,apple,341.06,538408569,90410870-9d8b-4b94-bc4b-a8ea0fb4cbbf
310,2020-01-01 00:20:35 UTC,view,4804718,2232732079706079299,sport bicycle,apple,341.06,547682192,68f6ab2d-0148-4df8-bbfe-562bad5b8fa1
320,2020-01-01 00:21:26 UTC,view,4804718,2232732079706079299,sport bicycle,apple,341.06,595417348,bb8d05b9-7138-4de4-ab29-a6a56bc96944
...,...,...,...,...,...,...,...,...,...
5597695,2020-01-31 22:44:02 UTC,view,4804718,2232732079706079299,sport bicycle,apple,313.23,590706558,4bcac42e-d0cc-42c3-889e-8a14648ac51c
5597706,2020-01-31 22:44:28 UTC,view,4804718,2232732079706079299,sport bicycle,apple,313.23,513718301,0e829b05-17f6-483f-8e0c-310629491025
5597716,2020-01-31 22:44:49 UTC,view,4804718,2232732079706079299,sport bicycle,apple,313.23,608810944,81016943-17b1-4a1a-8c74-95c6a826379d
5597961,2020-01-31 22:54:46 UTC,view,4804718,2232732079706079299,sport bicycle,apple,313.23,579782620,cb06500f-b73b-4b1c-9520-17b2939dd97b


In [8]:
X_pric = pricing_criterion(X_prep)

In [9]:
X_rating = rating(X_pric)

In [10]:
def metadata(X):
    X['metadata'] = X[['category_code', 'brand', 'price_category']].apply(lambda x: ' '.join(x), axis = 1)
    return X

In [11]:
X_meta = metadata(X_pric)

In [12]:
X_meta.query("brand == 'apple' and category_code == 'sport bicycle'")
# df_m.query("user_session == 'bde905b5-20f1-40ff-8ef6-a60d875e31e0'")

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,25%,75%,price_category,rating,metadata
4242570,2020-01-01 00:01:54 UTC,view,4802297,2232732079706079299,sport bicycle,apple,32.18,595414788,76885671-d2ed-4a44-b7a8-26733d77e095,26.63,164.22,medium,1,sport bicycle apple medium
4242571,2020-01-01 00:02:25 UTC,view,4804718,2232732079706079299,sport bicycle,apple,341.06,513814816,3d6aaddf-76a7-480f-af54-79b3a8ba14c4,26.63,164.22,high,1,sport bicycle apple high
4242582,2020-01-01 00:13:17 UTC,view,4804718,2232732079706079299,sport bicycle,apple,341.06,537104810,3c3c2ddd-a1ec-46cd-8d21-894872ce9be4,26.63,164.22,high,1,sport bicycle apple high
4242583,2020-01-01 00:14:22 UTC,view,4804056,2232732079706079299,sport bicycle,apple,164.28,595281688,a9983efc-028f-44e6-bf0d-1f65533d631c,26.63,164.22,high,1,sport bicycle apple high
4242584,2020-01-01 00:14:25 UTC,view,4804055,2232732079706079299,sport bicycle,apple,197.28,546206979,ca21825c-c359-44db-b1f3-536d50ceb6cd,26.63,164.22,high,1,sport bicycle apple high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4439192,2020-01-31 23:49:18 UTC,view,4804056,2232732079706079299,sport bicycle,apple,150.75,561065572,9ff4880f-b754-4c3a-aa74-d0a44e00862d,26.63,164.22,medium,1,sport bicycle apple medium
4439194,2020-01-31 23:49:58 UTC,view,4804056,2232732079706079299,sport bicycle,apple,150.75,561065572,9ff4880f-b754-4c3a-aa74-d0a44e00862d,26.63,164.22,medium,1,sport bicycle apple medium
4439195,2020-01-31 23:50:32 UTC,view,4804056,2232732079706079299,sport bicycle,apple,150.75,585422628,6daa1376-45ed-4d6d-b980-5110c91aa5db,26.63,164.22,medium,1,sport bicycle apple medium
4439196,2020-01-31 23:52:52 UTC,view,4804056,2232732079706079299,sport bicycle,apple,150.75,513111093,8f094079-5ae6-440c-bb3f-6f9c17d7425f,26.63,164.22,medium,1,sport bicycle apple medium


In [13]:
def ranking(X, n=5):
    X = pd.DataFrame(X_rating.groupby(by='product_id').rating.sum()).sort_values('rating', ascending=False).reset_index().index[0:n]
    return X
    

In [14]:
# def top_n(product_id, X_rating, n=10):
    
#     """select top n products by ranking'"""
    
#     rating_idx = pd.DataFrame(X_rating.groupby(by='product_id').rating.sum()).sort_values('rating', ascending=False).reset_index().index[0:n]
#     return rating_idx

#     counter = 0
    
#     for i in rating_idx:
#         meta_text = X_rating[X_rating['product_id'] == i][['metadata']].iloc[0,:][0]
        
#         if counter == 0:
#             print(f"Top {n} recommendations for product_id {product_id}:")
#             print(f"{i} - {meta_text} \n")
# #             print("-----------------------")

#         else:
#             print(f"Rec {counter}) {i} - {meta_text}")
        
#         counter += 1

In [15]:
def top_n_overall(product_id, X_rating, n=5):
    
    """select top n products by ranking'"""
    
    rating_idx = pd.DataFrame(X_rating.groupby(by='product_id').rating.sum()).sort_values('rating', ascending=False).index[0:n]
    #     rec_df.sort_values(ranking, ascending=False).index[0:n]
    counter = 0
    
    for i in rating_idx:
        meta_text = X_meta[X_meta['product_id'] == i][['metadata']].iloc[0,:][0]
        
        if counter == 0:
            print(f"Top {n} recommendations for product_id {product_id}:")
            print(f"{i} - {meta_text} \n")
#             print("-----------------------")

        else:
            print(f"Rec {counter}) {i} - {meta_text}")
        
        counter += 1

In [16]:
top_n_overall(49800017, X_rating)

Top 5 recommendations for product_id 49800017:
1004767 - construction tools light samsung medium 

Rec 1) 1005115 - construction tools light apple high
Rec 2) 1005100 - construction tools light samsung low
Rec 3) 1005160 - construction tools light xiaomi medium
Rec 4) 4804056 - sport bicycle apple high


In [17]:
def top_n_brands(X_rating, n=20):
    
    """select top n products by ranking'"""
    
    rating_idx =     pd.DataFrame(X_rating.groupby(['brand']).rating.sum()).sort_values('rating', ascending=False).index[0:n]

    #     rec_df.sort_values(ranking, ascending=False).index[0:n]
    return list(rating_idx)

In [18]:
top_n_brands(X_rating)

['samsung',
 'apple',
 'xiaomi',
 'huawei',
 'lucente',
 'sony',
 'lg',
 'oppo',
 'lenovo',
 'acer',
 'bosch',
 'hp',
 'asus',
 'artel',
 'casio',
 'dauscher',
 'redmond',
 'indesit',
 'respect',
 'nokia']

In [19]:
def top_n_by_brand(X_rating, brand, n=10):
    
    """select top n products by brand'"""
    
    rating_idx = pd.DataFrame(X_rating.groupby(['product_id','brand']).rating.sum()).sort_values('rating', ascending=False).reset_index('brand')
    rating_idx = rating_idx[rating_idx['brand']==brand].index[0:n]
#     return rating_idx

    counter = 0

    for i in rating_idx:
            meta_text = X_meta[X_meta['product_id'] == i][['metadata']].iloc[0,:][0]

            if counter == 0:
                print(f"Top {n} recommendations for product_id {brand}:")
                print(f"{i} - {meta_text} \n")
    #             print("-----------------------")

            else:
                print(f"Rec {counter}) {i} - {meta_text}")

            counter += 1

In [20]:
top_n_by_brand(X_rating, brand='apple')

Top 10 recommendations for product_id apple:
1005115 - construction tools light apple high 

Rec 1) 4804056 - sport bicycle apple high
Rec 2) 1002544 - construction tools light apple medium
Rec 3) 1005105 - construction tools light apple high
Rec 4) 1004249 - construction tools light apple high
Rec 5) 4804718 - sport bicycle apple high
Rec 6) 1005116 - construction tools light apple high
Rec 7) 4804055 - sport bicycle apple high
Rec 8) 1002524 - construction tools light apple medium
Rec 9) 1005135 - construction tools light apple high
